# Mid-Term Project

Problem Statement: Predict whether patients will have reccurant events

In [2]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
# fetch dataset, save as breast cancer dataframe (bcdf)
column_labels = ['Class','age','menopause','tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat']
bcdf = pd.read_csv('breast-cancer.data')
bcdf.columns = column_labels

### Exploratory Data Analysis

In [15]:
bcdf.shape

(285, 10)

In [11]:
bcdf.head()

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
2,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
3,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no
4,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,left,left_low,no


In [13]:
bcdf.describe(include='all')

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
count,285,285,285,285,285,285,285.000000,285,285,285
unique,2,6,3,11,7,3,NaN,2,6,2
top,no-recurrence-events,50-59,premeno,30-34,0-2,no,NaN,left,left_low,no
freq,200,96,149,59,212,221,NaN,151,109,217
mean,NaN,NaN,NaN,NaN,NaN,NaN,2.045614,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,0.737351,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN


In [20]:
for column in column_labels:
    print(column, bcdf[column].unique())

Class ['no-recurrence-events' 'recurrence-events']
age ['40-49' '60-69' '50-59' '30-39' '70-79' '20-29']
menopause ['premeno' 'ge40' 'lt40']
tumor-size ['20-24' '15-19' '0-4' '25-29' '50-54' '10-14' '30-34' '40-44' '35-39'
 '5-9' '45-49']
inv-nodes ['0-2' '6-8' '9-11' '3-5' '15-17' '12-14' '24-26']
node-caps ['no' 'yes' '?']
deg-malig [2 1 3]
breast ['right' 'left']
breast-quad ['right_up' 'left_low' 'left_up' 'right_low' 'central' '?']
irradiat ['no' 'yes']


In this dataset, '?' signifies missing data, so calling .unique() will display which columns have missing data (if any).  Columns 'node-caps' and 'breast-quad' have missing data.  Fill in missing data with data that is most common.  For 'node-caps' attribute, the most common value is 'no' with 221 out of 285 being 'no'.  For 'breast-quad' attribute, the most common value is 'low_left' with 109 out of 285 being 'low_left'.

In [26]:
bcdf['node-caps'].replace(to_replace='?', value='no', inplace=True)
bcdf['breast-quad'].replace(to_replace='?', value='left_low', inplace=True)

In [27]:
bcdf.describe(include='all')

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
count,285,285,285,285,285,285,285.000000,285,285,285
unique,2,6,3,11,7,2,NaN,2,5,2
top,no-recurrence-events,50-59,premeno,30-34,0-2,no,NaN,left,left_low,no
freq,200,96,149,59,212,229,NaN,151,110,217
mean,NaN,NaN,NaN,NaN,NaN,NaN,2.045614,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,0.737351,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN


In [28]:
for column in column_labels:
    print(column, bcdf[column].unique())

Class ['no-recurrence-events' 'recurrence-events']
age ['40-49' '60-69' '50-59' '30-39' '70-79' '20-29']
menopause ['premeno' 'ge40' 'lt40']
tumor-size ['20-24' '15-19' '0-4' '25-29' '50-54' '10-14' '30-34' '40-44' '35-39'
 '5-9' '45-49']
inv-nodes ['0-2' '6-8' '9-11' '3-5' '15-17' '12-14' '24-26']
node-caps ['no' 'yes']
deg-malig [2 1 3]
breast ['right' 'left']
breast-quad ['right_up' 'left_low' 'left_up' 'right_low' 'central']
irradiat ['no' 'yes']


There is no more missing data. Next I need to replace the categorical data with numerical data.  The 'Class', 'node-cap', 'breast', and 'irradiat' attributes can be binarized. 

In [39]:
bcdf['Class'].replace(to_replace='no-recurrence-events', value='0', inplace=True)
bcdf['Class'].replace(to_replace='recurrence-events', value='1', inplace=True)

bcdf['node-caps'].replace(to_replace='no', value='0', inplace=True)
bcdf['node-caps'].replace(to_replace='yes', value='1', inplace=True)

bcdf['breast'].replace(to_replace='right', value='0', inplace=True)
bcdf['breast'].replace(to_replace='left', value='1', inplace=True)

bcdf['irradiat'].replace(to_replace='no', value='0', inplace=True)
bcdf['irradiat'].replace(to_replace='yes', value='1', inplace=True)

In [40]:
bcdf.head(30)

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,0,40-49,premeno,20-24,0-2,0,2,0,right_up,0
1,0,40-49,premeno,20-24,0-2,0,2,1,left_low,0
2,0,60-69,ge40,15-19,0-2,0,2,0,left_up,0
3,0,40-49,premeno,0-4,0-2,0,2,0,right_low,0
4,0,60-69,ge40,15-19,0-2,0,2,1,left_low,0
5,0,50-59,premeno,25-29,0-2,0,2,1,left_low,0
6,0,60-69,ge40,20-24,0-2,0,1,1,left_low,0
7,0,40-49,premeno,50-54,0-2,0,2,1,left_low,0
8,0,40-49,premeno,20-24,0-2,0,2,0,left_up,0
9,0,40-49,premeno,0-4,0-2,0,3,1,central,0
